DC2 Retrieve Forced Src catalogs of visit-level forced photoemtry from coadd Object catalog.  
Michael Wood-Vasey

This Notebook shows how to retrieve the list of forced sources from a given tract, patch, filter.  
The forced-source dataIds contain sufficient information to look up the calexp IDs to retrieve postage stamps.

In [ ]:
import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage

In [ ]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
butler = dafPersist.Butler(repo)

In [ ]:
tract = 4850
patch = '5,6'

filt = 'r'
partial_data_id = {'tract': tract, 'patch': patch, 'filter': filt}

In [ ]:
dataset_type = 'forced_src'
subset = butler.subset(datasetType=dataset_type, dataId=partial_data_id)
data_ids = [dr.dataId for dr in subset
           if butler.datasetExists(datasetType=dataset_type,
                                   dataId=dr.dataId)]

In [ ]:
print(data_ids[0:3])

In [ ]:
dId = {'tract': 4848, 'patch': '5,6', 'filter': 'r', 'visit': 193861, 'raft': '1,4', 'sensor': '0,0'}

calibrated_exposure = butler.get(datasetType='calexp', dataId=dId)
forced_src = butler.get(datasetType=dataset_type, dataId=dId)

In [ ]:
cat = forced_src.asAstropy()

In [ ]:
calib = calibrated_exposure.getCalib()
calib.setThrowOnNegativeFlux(False)

mag, mag_err = calib.getMagnitude(forced_src['base_PsfFlux_flux'],
                                  forced_src['base_PsfFlux_fluxSigma'])
cat['mag'] = mag
cat['mag_err'] = mag_err
cat['snr'] = np.abs(cat['base_PsfFlux_flux'])/cat['base_PsfFlux_fluxSigma']

In [ ]:
print(cat)

This is per tract, so 22,062 sources seems potentially reasonble.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
plt.scatter(np.rad2deg(cat['coord_ra']),
            np.rad2deg(cat['coord_dec']))
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')

In [ ]:
plt.scatter(cat['mag'], cat['snr'])
plt.xlabel('%s mag' % dId['filter'])
plt.ylabel('S/N')


Looks like a reasonable set of magnitudes and uncertainties.